In [1]:
import pandas as pd
import numpy as np
import numbers
import operator
import re

In [2]:
ID_var_name = 'pet_mr_radiomics'

df_yr = range(1,6,1)
outcome_names = ['DF_{}yr'.format(yy) for yy in df_yr]

n_trial = 1000
k_fold = 3

rootdir = '/Users/shuang/Documents/Proj_Radiomics/Data/her2'
the_mri_tp = 2
the_mri_bin_width = 5
the_pet_bin_width = 0.1
im_dir = '{}/her2_Analysis/PETMRI/PETbinwidth{:.1f}_MRItp{}_binwidth{}'.format(rootdir,the_pet_bin_width, the_mri_tp, the_mri_bin_width)

# the selected model to use for all DF outcome
the_clf_name = 'ElasticNet'

In [5]:
# find 'important' features
for oc in outcome_names:
    json_fname = '{}/Learner/{}_IDV{}_DV{}_Trial{}_{}folds.json'.format(im_dir, the_clf_name, ID_var_name, oc, n_trial, k_fold)
    df_learning_output = pd.read_json(json_fname)
    for ii in df_learning_output.index:
        feat_imp = np.array(df_learning_output['feat_importance'][ii])
        feat_names = np.array(df_learning_output['feat_name'][ii])
        
        # find a list of features that are not dropped by ElasticNet
        lst_tmp = feat_names[feat_imp != 0.0]
        if ii == 0:
            #initialize the dictionary
            dct_feat_tally = {}
            for ss in lst_tmp:
                dct_feat_tally[ss] = 1
        else:
            for ss in lst_tmp:
                if ss in dct_feat_tally.keys():
                    dct_feat_tally[ss] += 1
                else:
                    dct_feat_tally[ss] = 1
    
    # sort the feat tally dictionary
    sorted_dct_feat_tally = sorted(dct_feat_tally.items(), key=operator.itemgetter(1), reverse=True)
    top10_feat = sorted_dct_feat_tally[0:10]
    
    max_count = len(df_learning_output)
    print('outcome: {}\n'.format(oc))
    for (feat, count) in top10_feat:
        print('{} ({})'.format(feat, float(count/max_count)))
    print('\n')
#     print(sorted_dct_feat_tally)
#     for k, value in sorted_dct_feat_tally.items():
#         print('feat name: {}, count: {}'.format(k, value))
    
    

outcome: DF_1yr

texture_idn_avg_mri (0.991)
texture_idmn_avg_mri (0.841)
texture_cluster_prominence_avg_pet (0.83)
FOstats_entropy_mri (0.815)
FOstats_mean_mri (0.775)
texture_sum_entropy_avg_mri (0.7623333333333333)
texture_sum_avg_avg_mri (0.7466666666666667)
texture_avg_intensity_avg_mri (0.7466666666666667)
FOstats_min_mri (0.7386666666666667)
texture_diff_entropy_avg_mri (0.72)


outcome: DF_2yr

FOstats_mean_mri (0.9816666666666667)
texture_sum_avg_avg_mri (0.9806666666666667)
texture_avg_intensity_avg_mri (0.9806666666666667)
FOstats_min_mri (0.9663333333333334)
FOstats_max_mri (0.894)
texture_idn_avg_mri (0.8746666666666667)
texture_diff_avg_avg_mri (0.8713333333333333)
texture_dissimilarity_avg_mri (0.8713333333333333)
FOstats_min_pet (0.8626666666666667)
ShapeSize_compactness2_mri (0.843)


outcome: DF_3yr

FOstats_mean_mri (0.9893333333333333)
texture_sum_avg_avg_mri (0.9836666666666667)
texture_avg_intensity_avg_mri (0.9836666666666667)
FOstats_min_mri (0.968)
texture_diff

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, auc, roc_curve
from scipy.stats import zscore

In [6]:
# read in all radiomic data
fname = '{}/data_all.csv'.format(im_dir)
df_data = pd.read_csv(fname)
pat = re.compile('texture_|FOstats_|ShapeSize_')
feat_names = [ss for ss in df_data.columns.tolist() if pat.match(ss)]
feat_tag = 'pet_mr_radiomics'

# scale the features to z-score
df_data[feat_names] = df_data[feat_names].apply(zscore)

In [10]:
len(df_data.MRN.unique())

113

In [ ]:
#read the learning output data and more
fname = '{}/Learner/CLF_output_all.csv'.format(im_dir)
df_clf_all = pd.read_csv(fname)

# simplify clf name
def simplify_clf_name_func(x):
    if re.search('(.+)LogReg',x):
        return 'LogReg'
    else:
        return x
df_clf_all['clf_name_simplify'] = df_clf_all['clf_name'].map(simplify_clf_name_func)
# print(df_clf_all.ix[:,['clf_name_simplify', 'clf_name']])

# # drop the one with Random Forest since we would like to use logistic regression only
# df_clf_all.drop(df_clf_all[(df_clf_all.Dep_var == 'DF_1yr') & (df_clf_all.clf_name == 'RandomForest')].index, inplace=True)
# idx = df_clf_all.groupby('Dep_var').apply(lambda df: df.AUC_mean.argmax())  

dct_clf = {'ElasticNet': SGDClassifier(), 'LogReg': LogisticRegression(), 'RandomForest': RandomForestClassifier(),
          'SVM': SVC()}


In [75]:
test = [idx[3]]
print(test)
for ii in test:
#     clf_name, clf_name_simple, oc, n_trial, k_fold = df_clf_all.ix[ii, ['clf_name', 'clf_name_simplify', 'Dep_var', 'n_trial', 'k_fold']].tolist()
    json_fname = '{}/Learner/{}_IDV{}_DV{}_Trial{}_{}folds.json'.format(im_dir, the_clf_name, feat_name, oc, n_trial, k_fold)
    df_learning_output = pd.read_json(json_fname)
    lst_data_all = []
    for iid in df_learning_output.index:
        dct_param = df_learning_output['best_params'][iid]
        if iid == 0: # determine what to average on the first round
            lst_num_var = []
            for k, val in dct_param.items():
               if isinstance(val, numbers.Number):
                lst_num_var.append(k)
            lst_str_var = set(lst_num_var).symmetric_difference(dct_param.keys())
        lst_tmp = [dct_param[s] for s in lst_num_var]
        tmp = dict(zip(lst_num_var, lst_tmp))
        lst_data_all.append(tmp)
        
    df_data_all = pd.DataFrame(lst_data_all)
    df_data_all = df_data_all.ix[:, lst_num_var]
    
    # create the final parameter with the average over all numerical variables
    final_param = dict(df_data_all.mean())
    
    # add the string parameter variabls
    for ss in lst_str_var:
        final_param[ss] = dct_param[ss]
    print(final_param)
        
    # get all the needed radiomic data
    df_tmp = df_data.ix[:,feat_names + [oc]]
    df_tmp = df_tmp.dropna()
    X, y = df_tmp.ix[:,feat_names].as_matrix(), df_tmp.ix[:,oc].astype('int').as_matrix()
    clf = dct_clf[the_clf_name]
    print(clf)
    clf.set_params(**final_param)
    clf.fit(X, y)
    feat_importance = getattr(clf, 'feature_importances_', None)
    if feat_importance is None and hasattr(clf, 'coef_'):
        feat_importance = clf.coef_
    print(feat_importance)


[31]
/Users/shuang/Documents/Proj_Radiomics/Data/her2/her2_Analysis/PETMRI/PETbinwidth0.1_MRItp2_binwidth5/Learner/ElasticNet_IDVpet_mr_radiomics_DVDF_3yr_Trial10_3folds.json
{'alpha': 0.68797000000000097, 'l1_ratio': 0.48666666666666664, 'max_iter': 49933.333333333336, 'loss': 'log', 'penalty': 'elasticnet'}
SGDClassifier(alpha=0.68797000000000097, average=False, class_weight=None,
       epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.48666666666666664, learning_rate='optimal', loss='log',
       max_iter=49933.333333333336, n_iter=None, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       tol=None, verbose=0, warm_start=False)
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. 

In [62]:
feat_imp = np.array(df_learning_output['feat_importance'][4])
feat_name = np.array(df_learning_output['feat_name'][4])
print(feat_name[feat_imp != 0.0])
print(feat_imp)

['texture_idn_avg_mri' 'texture_cluster_shade_avg_pet']
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.26841557  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.18527969
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.        ]


In [15]:
# for easier interpretbaility of output, we will use elasticNet for reporting feature importance for all outcome!

Object `OrderedDict` not found.


[('texture_imc2_avg_mri', 25),
 ('FOstats_mean_mri', 25),
 ('texture_idn_avg_pet', 25),
 ('texture_idn_avg_mri', 24),
 ('ShapeSize_max_euc_dis_pet', 24),
 ('texture_avg_intensity_avg_mri', 24),
 ('texture_sum_avg_avg_mri', 24),
 ('texture_cluster_prominence_avg_pet', 21),
 ('texture_diff_avg_avg_mri', 20)]